# Descubrimiento de temas con K-Means

En este notebook se identifican temas principales en los mensajes
utilizando K-Means sobre representaciones TF-IDF.


In [1]:
import pandas as pd #armar y guardar el CSV
import pickle #cargar modelos y matrices guardadas
import numpy as np
from sklearn.cluster import KMeans #algoritmo de clustering

**Función balance_temas**
* Cuenta cuántos mensajes hay por tema (value_counts)
* Calcula la proporción de cada tema
* Sirve para ver si los clusters están balanceados o dominados por uno

In [2]:
def balance_temas(df, col="topic"):
    conteo = df[col].value_counts().sort_index()
    prop = (df[col].value_counts(normalize=True).sort_index()).round(3)

    print("\nBalance de temas (conteo):")
    print(conteo)
    print("\nProporciones:")
    print(prop)

**Función top_terminos_por_tema**
* ordena el centroide del cluster Y selecciona los términos con mayor peso TF-IDF
* Muestra las palabras más representativas de cada tema

In [3]:
def top_terminos_por_tema(kmeans, feature_names, n=12): #Modelo Kmeans ya entrenado, lista de palabras IF-IDF
    for t in range(kmeans.n_clusters):
        idx = kmeans.cluster_centers_[t].argsort()[::-1][:n]
        print(f"\nTema {t} - top términos:")
        print(", ".join(feature_names[idx]))

**Función ejemplos_por_tema**
* Muestra ejemplos reales de mensajes por tema
* Ayuda a entender qué tipo de texto cayó en cada cluster
* Complementa los términos con contexto real

In [4]:
def ejemplos_por_tema(df, k, n_ejemplos=3, max_chars=220): #DataFrame que contien los textos y las columnas
    #k numero de temas, números de ejemplos 3, máximo de caracteres por ejemplo
    for t in range(k): #Entra 5
        print(f"\n--- Tema {t} (ejemplos) ---")
        textos = df[df["topic"] == t]["text_clean"].head(n_ejemplos).tolist()
        for i, txt in enumerate(textos, 1):
            print(f"{i}. {txt[:max_chars]}{'...' if len(txt) > max_chars else ''}")

# REPO AUTO1111_WEBUI

In [5]:
slug = "auto1111_webui"
k = 5 #Número de temas

# Cargar datos
df = pd.read_csv(f"../data/{slug}/messages_clean_ready.csv") #ruta donde está el archivo generado en el file 3

with open(f"../data/{slug}/X_tfidf.pkl", "rb") as f: #Carga de la matriz TF - IDF númerica de texto
    X_tfidf = pickle.load(f)

with open(f"../data/{slug}/tfidf_vectorizer.pkl", "rb") as f: #recupera el vector ya entrenado con el vocabulario
    vectorizer = pickle.load(f)

feature_names = np.array(vectorizer.get_feature_names_out()) # Obtener nombres de términos

# Entrenar KMeans
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10) #KMeans intenta 10 inicializaciones distintas y se queda con la mejor
kmeans.fit(X_tfidf) #Entrena el modelo

df["topic"] = kmeans.labels_ #Asignar el tema a cada texto

print("REPO:", slug)
balance_temas(df, "topic")

REPO: auto1111_webui

Balance de temas (conteo):
topic
0    110
1     28
2    270
3     29
4     22
Name: count, dtype: int64

Proporciones:
topic
0    0.240
1    0.061
2    0.588
3    0.063
4    0.048
Name: proportion, dtype: float64


In [6]:
top_terminos_por_tema(kmeans, feature_names, n=12) #Presentamos los temas con las palabras top


Tema 0 - top términos:
webui, diffusion, stable diffusion, stable, diffusion webui, issue, py, file, line, py line, issue exists, webui issue

Tema 1 - top términos:
feature, existing, would, would feature, issue searched, issues checked, existing issues, existing issue, proposed workflow, checked recent, commits, recent builds

Tema 2 - top términos:
issue, python, using, torch, sqlite, version, install, gpu, also, venv, webui, try

Tema 3 - top términos:
python, pip, clip, pip install, exe pip, install, python exe, exe, webui, isolation, build isolation, sd webui

Tema 4 - top términos:
hash, commit, repo, commit hash, url, eb, change url, hash eb, set stable, change, repository, stable


In [7]:
ejemplos_por_tema(df, k, n_ejemplos=3) #Presentamos ejemplos en formato frase


--- Tema 0 (ejemplos) ---
1. bug runtimeerror clone stable diffusion checklist x issue exists disabling extensions x issue exists clean installation webui issue caused extension believe caused bug webui x issue exists current version webui x issue r...
2. open modules launch utils py edit stable diffusion repo os environ get stable diffusion repo stable diffusion repo os environ get stable diffusion repo
3. similar issue im issues code lines also different install clicking webui user bat looks like log venv c ai stable diffusion webui venv scripts python exe python tags v c b bd aug msc v bit amd version v commit hash c ae ...

--- Tema 1 (ejemplos) ---
1. also facing issue img width height alt image src
2. feature request sort options settings automating various things possible existing issue x searched existing issues checked recent builds commits would feature feature enhancement request could possibly get sort options s...
3. feature request appimage existing issue x searched exis

Tema	Interpretación

0	Bugs y errores internos del WebUI

1	Solicitudes de nuevas funcionalidades

2	Problemas generales de instalación

3	Errores técnicos con dependencias y pip

4	Problemas de repositorio y commit hash

In [8]:
# Guardar resultados por repo
df.to_csv(f"../data/{slug}/messages_with_topics.csv", index=False)
with open(f"../data/{slug}/kmeans_model.pkl", "wb") as f:
    pickle.dump(kmeans, f)

print("\nGuardado  messages_with_topics.csv + kmeans_model.pkl")


Guardado  messages_with_topics.csv + kmeans_model.pkl


# REPO YTDLP

In [9]:
slug = "ytdlp"
k = 5

df = pd.read_csv(f"../data/{slug}/messages_clean_ready.csv")

with open(f"../data/{slug}/X_tfidf.pkl", "rb") as f:
    X_tfidf = pickle.load(f)

with open(f"../data/{slug}/tfidf_vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

feature_names = np.array(vectorizer.get_feature_names_out())

kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
kmeans.fit(X_tfidf)

df["topic"] = kmeans.labels_

print("REPO:", slug)
balance_temas(df, "topic")

REPO: ytdlp

Balance de temas (conteo):
topic
0    111
1     36
2    396
3     95
4     59
Name: count, dtype: int64

Proporciones:
topic
0    0.159
1    0.052
2    0.568
3    0.136
4    0.085
Name: proportion, dtype: float64


In [10]:
top_terminos_por_tema(kmeans, feature_names, n=12)


Tema 0 - top términos:
debug, yt dlp, dlp, yt, line, utf, py line, file, py, generic, command, file yt

Tema 1 - top términos:
mp, dash, avc, mib, mp mib, mp dash, mib avc, andr, debug, audio, video, webm

Tema 2 - top términos:
also, would, drm, extractor, video, yt, yes, url, like, yt dlp, dlp, mean

Tema 3 - top términos:
debug, yt, dlp, yt dlp, youtube, downloading, download, mp, cookies, bytevc, audio, format

Tema 4 - top términos:
issue, still, open, verbose log, pr, could, like, cookies, related, log, work, firefox


In [11]:
ejemplos_por_tema(df, k, n_ejemplos=3)


--- Tema 0 (ejemplos) ---
1. error yandexvideopreview unable extract data raw checklist x reporting yt dlp broken supported site x verified updated yt dlp nightly master update instructions x checked provided urls playable browser ip login details x...
2. applepodcasts json embedded episode html podcasts apple com changed checklist x reporting yt dlp broken supported site x verified updated yt dlp nightly master update instructions x checked provided urls playable browser...
3. pr restores applepodcasts functionality commit fcc b ie current head commit pr branch git describe tags gd fcc b applepodcasts works venv bin yt dlp vu debug command line config vu debug encodings locale utf fs utf pref ...

--- Tema 1 (ejemplos) ---
1. verbose open pr code cleanup giving verbose correct since site supported details summary series verbose summary cmd ertflix extracting url ertflix ert ds e downloading json metadata ertflix ert ds downloading json metada...
2. ff diagnostic flagged unsupported d

Tema	Interpretación

0	Errores en extractores y parsing

1	Formatos multimedia y DRM

2	Opciones avanzadas del CLI

3	Descarga de YouTube y cookies

4	Reportes de bugs y logs

Guardar resultados por repo

In [13]:
# Guardar resultados por repo
df.to_csv(f"../data/{slug}/messages_with_topics.csv", index=False)
with open(f"../data/{slug}/kmeans_model.pkl", "wb") as f:
    pickle.dump(kmeans, f)

print("\nGuardado messages_with_topics.csv + kmeans_model.pkl")


Guardado messages_with_topics.csv + kmeans_model.pkl
